# 🚀 Day 1-4:  GBMs (Gradient Boosting Machines)
---
## Ⅰ. Gradient Boosting 개론

앙상블(Ensemble) 기법은 여러 개의 약한 학습기(Weak Learner)를 결합하여 하나의 강력한 학습기(Strong Learner)를 만드는 방법론입니다. 

그중에서도 **Gradient Boosting**은 이전 모델의 오차(Error)를 다음 모델이 학습하여 점진적으로 성능을 개선해 나가는 방식으로, 예측 성능이 매우 뛰어나 많은 데이터 분석 대회에서 우승 모델의 기반이 되었습니다.

---

### 1. 핵심 개념: 오차를 보완하며 똑똑해지기 🧠

Gradient Boosting의 아이디어는 매우 직관적입니다.

1.  **첫 번째 예측**: 먼저 아주 간단한 모델(예: 타겟 변수의 평균값)로 예측을 시작합니다. 당연히 이 예측은 실제 값과 큰 차이를 보일 것입니다. 이 차이를 **잔차(Residual)** 라고 합니다.

2.  **잔차 학습**: 새로운 모델(주로 결정 트리)을 학습시킵니다. 단, 이때 모델은 실제 값을 예측하는 것이 아니라, 앞에서 발생한 **잔차를 예측** 하도록 학습됩니다.
3.  **예측 업데이트**: 기존 예측값에 '학습률(Learning Rate)'을 곱한 '잔차 예측값'을 더하여 예측을 업데이트합니다. 학습률은 새로운 모델이 학습한 내용을 얼마나 반영할지 결정하는 스텝 사이즈로, 너무 크면 과적합의 위험이 있습니다.
      - $New\_Prediction = Old\_Prediction + LearningRate \times Residual\_Prediction$
4.  **반복**: 새로운 예측값에 대한 잔차를 다시 계산하고, 이 잔차를 학습하는 또 다른 모델을 추가합니다. 이 과정을 지정된 횟수만큼 반복합니다.

이처럼 각 단계의 모델은 이전 단계 모델의 부족한 부분을 보완하는 방향으로 학습하기 때문에, 모델이 점진적으로 정답에 가까워지게 됩니다. 이를 **점진적 학습(Stage-wise Additive Modeling)** 이라고 부릅니다.

---

### 2. `scikit-learn`으로 Gradient Boosting 구현하기

Python의 대표적인 머신러닝 라이브러리 `scikit-learn`은 `GradientBoostingRegressor`를 제공하여 이 개념을 쉽게 구현할 수 있게 해줍니다.

'서울 따릉이' 데이터를 사용해 시간대별 자전거 대여량을 예측하는 회귀 모델을 만들어 보겠습니다.

#### 데이터 준비

먼저 필요한 라이브러리를 가져오고 데이터를 불러와 간단한 전처리를 수행합니다.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import plotly.express as px

데이터 준비

In [4]:
# 데이터 로드 (파일 경로는 환경에 맞게 조정해주세요)
df = pd.read_csv('../datasets/ml/bike-sharing/SeoulBikeData.csv', encoding='cp949')

# 컬럼명 영문으로 변경 및 공백 제거
df.columns = ['Date', 'Rented_Bike_Count', 'Hour', 'Temperature', 'Humidity',
              'Wind_Speed', 'Visibility', 'Dew_Point_Temp', 'Solar_Radiation',
              'Rainfall', 'Snowfall', 'Seasons', 'Holiday', 'Functioning_Day']

# 데이터 타입 변경 및 파생 변수 생성
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Weekday'] = df['Date'].dt.day_name()

# 범주형 변수 원-핫 인코딩
df = pd.get_dummies(df, columns=['Seasons', 'Holiday', 'Functioning_Day', 'Weekday'], drop_first=True)

# 불필요한 컬럼 제거
df = df.drop(['Date'], axis=1)

데이터 분리

In [5]:
# 특성과 타겟 분리
X = df.drop('Rented_Bike_Count', axis=1)
y = df['Rented_Bike_Count']

# 학습 및 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("데이터 준비 완료!")
print(f"학습 데이터 형태: {X_train.shape}")
print(f"테스트 데이터 형태: {X_test.shape}")

데이터 준비 완료!
학습 데이터 형태: (7008, 23)
테스트 데이터 형태: (1752, 23)


#### 모델 학습 및 평가

`GradientBoostingRegressor`의 주요 하이퍼파라미터는 다음과 같습니다.

  * `n_estimators`: 부스팅 단계를 몇 번 반복할지 (즉, 몇 개의 트리를 만들지) 결정합니다.
  * `learning_rate`: 학습률입니다. 각 트리의 기여도를 조절하며, 낮을수록 안정적이지만 `n_estimators`를 늘려야 합니다.
  * `max_depth`: 각 결정 트리의 최대 깊이를 제한하여 모델의 복잡도를 조절합니다.
  * `subsample`: 전체 학습 데이터 중 일부만 샘플링하여 각 트리를 학습시키는 비율입니다. 과적합 방지에 효과적입니다. (Stochastic Gradient Boosting)

<!-- end list -->

In [6]:
# Gradient Boosting 모델 초기화 및 학습
gb_reg = GradientBoostingRegressor(
    n_estimators=500,       # 500개의 트리 사용
    learning_rate=0.1,      # 학습률 0.1
    max_depth=4,            # 트리 최대 깊이 4
    subsample=0.8,          # 80%의 데이터만 샘플링하여 사용
    random_state=42,
    verbose=1               # 학습 과정 출력
)

gb_reg.fit(X_train, y_train)


      Iter       Train Loss      OOB Improve   Remaining Time 
         1      359774.4045       55244.2031           10.16s
         2      320038.8561       66977.8236            7.60s
         3      279292.7907       15250.7611            6.65s
         4      248685.1069       32071.1328            6.09s
         5      224826.9356       31398.9759            5.72s
         6      203527.3271       18716.6067            5.45s
         7      181183.5442       -3309.2218            5.22s
         8      168470.7043       22217.5169            5.05s
         9      158034.6030       24141.7170            4.90s
        10      147102.1493       13550.1528            4.78s
        20       87699.9671        6261.7630            4.24s
        30       66985.1897        3134.0864            4.01s
        40       56474.2820       -3076.4961            3.85s
        50       48408.0643       -6453.5899            3.74s
        60       45147.5475        8746.2438            3.62s
       

GradientBoostingRegressor(max_depth=4, n_estimators=500, random_state=42,
                          subsample=0.8, verbose=1)

In [7]:
# 예측 및 성능 평가
y_pred_gb = gb_reg.predict(X_test)
rmse_gb = np.sqrt(mean_squared_error(y_test, y_pred_gb))
r2_gb = r2_score(y_test, y_pred_gb)

print(f"[Gradient Boosting Regressor 성능]")
print(f"RMSE: {rmse_gb:.4f}")
print(f"R^2 Score: {r2_gb:.4f}")

[Gradient Boosting Regressor 성능]
RMSE: 168.0279
R^2 Score: 0.9322


### 3. 연습문제

문제 1. `GradientBoostingRegressor`의 `n_estimators`를 `100`으로, `learning_rate`를 `0.05`로 변경하여 모델을 다시 학습시키고, 성능(RMSE)이 어떻게 변하는지 확인해보세요.

문제 2. `learning_rate`와 `n_estimators`는 서로 어떤 관계가 있을까요? 만약 `learning_rate`를 매우 낮게 설정한다면, `n_estimators`는 어떻게 조절해야 비슷한 수준의 성능을 기대할 수 있을지 설명해보세요.

----
## Ⅱ. 고성능 Gradient Boosting 라이브러리 (XGBoost, LightGBM, CatBoost)

`scikit-learn`의 `GradientBoostingRegressor`는 개념을 이해하기에 훌륭하지만, 대용량 데이터에 대한 학습 속도나 성능 면에서는 한계가 있습니다. 이를 극복하기 위해 등장한 것이 **XGBoost, LightGBM, CatBoost**이며, 이들은 '부스팅 3대장'으로 불립니다.


### 1. XGBoost (eXtreme Gradient Boosting) 🚀

XGBoost는 Gradient Boosting 알고리즘을 병렬 처리, 규제(Regularization), 가지치기(Pruning) 등의 기법을 통해 극단(eXtreme)으로 개선한 라이브러리입니다. 

한동안 캐글과 같은 데이터 경진대회를 휩쓸며 명성을 떨쳤습니다.

#### XGBoost의 주요 특징

  * **병렬 처리**: 트리 노드를 분할할 때 여러 CPU 코어를 활용하여 학습 속도가 매우 빠릅니다.
  * **규제(Regularization)**: 모델이 과적합되는 것을 방지하기 위해 L1(`reg_alpha`), L2(`reg_lambda`) 규제 항을 손실 함수에 포함합니다.
  * **자체적인 교차 검증**: 모델 학습 시 내장된 `cv` 기능을 활용할 수 있습니다.
  * **결측치 처리**: 데이터에 결측치가 있어도 이를 자체적으로 처리하는 메커니즘을 가지고 있습니다.


#### 코드 예제

먼저 XGBoost 라이브러리를 설치해야 합니다.
`!pip install xgboost`

In [ ]:
!pip install xgboost

In [9]:
import xgboost as xgb

# XGBoost 모델 초기화 및 학습
xgb_reg = xgb.XGBRegressor(
    n_estimators=500,
    learning_rate=0.1,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8, # 각 트리를 만들 때 특성의 80%를 샘플링
    gamma=0.1,             # 분할에 필요한 최소 손실 감소량 (규제 효과)
    random_state=42,
    n_jobs=-1,              # 사용 가능한 모든 CPU 코어 사용
    early_stopping_rounds=50, # 50 라운드 동안 성능 개선 없으면 조기 종료
)

xgb_reg.fit(X_train, y_train,
            eval_set=[(X_test, y_test)], # 검증 세트 지정
            verbose=False)

# 예측 및 성능 평가
y_pred_xgb = xgb_reg.predict(X_test)
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
r2_xgb = r2_score(y_test, y_pred_xgb)

print(f"[XGBoost Regressor 성능]")
print(f"RMSE: {rmse_xgb:.4f}")
print(f"R^2 Score: {r2_xgb:.4f}")

[XGBoost Regressor 성능]
RMSE: 165.9502
R^2 Score: 0.9339


*Note: `colsample_bytree`는 각 트리를 구성할 때 전체 특성 중 일부만 랜덤하게 선택하는 기능으로, Random Forest의 아이디어와 유사하며 과적합 방지에 도움을 줍니다.*

#### 🎯 연습문제 3

XGBoost의 규제 파라미터인 `reg_alpha` (L1)와 `reg_lambda` (L2)의 역할은 무엇일까요? `reg_lambda=10`으로 설정하여 모델을 다시 학습시키고, 결과에 어떤 변화가 있는지 확인해보세요.


#### 🎯 연습문제 4

XGBoost 모델의 `feature_importances_` 속성을 사용하여 어떤 특성이 따릉이 수요 예측에 가장 중요하게 작용했는지 확인하고, `plotly.express`를 이용해 상위 10개 특성을 막대그래프로 시각화해보세요.


*해설: 보통 시간(Hour), 온도(Temperature), 계절(Seasons)과 관련된 특성들이 상위권에 위치할 가능성이 높습니다. 특성 중요도를 통해 모델이 어떤 정보를 기반으로 예측하는지 이해할 수 있습니다.*


-----

### 2. LightGBM (Light Gradient Boosting Machine) ⚡️

LightGBM은 Microsoft에서 개발한 부스팅 라이브러리로, XGBoost보다 더 빠른 학습 속도와 적은 메모리 사용량을 자랑합니다. 대용량 데이터셋에서 특히 강력한 성능을 발휘합니다.

#### LightGBM의 주요 특징

  * **리프 중심 트리 분할(Leaf-wise Tree Growth)**: 기존의 부스팅 모델들이 트리를 수평적으로 확장(Level-wise)하는 반면, LightGBM은 손실(loss)이 가장 크게 감소하는 방향으로 리프 노드를 선택하여 수직적으로 트리를 확장합니다. 이는 더 적은 노드로도 높은 성능을 낼 수 있게 해주어 학습 속도를 비약적으로 향상시킵니다.
      * **단점**: 데이터의 양이 적을 경우(수천 개 미만) 깊은 트리가 생성되어 과적합되기 쉽습니다.
  * **GOSS (Gradient-based One-Side Sampling)**: 기울기(gradient)가 큰 데이터(예측이 많이 틀린 데이터)에 더 집중하고, 기울기가 작은 데이터는 일부만 샘플링하여 학습 효율을 높입니다.
  * **EFB (Exclusive Feature Bundling)**: 동시에 0이 아닌 값을 거의 갖지 않는 희소한(sparse) 특성들(예: 원-핫 인코딩된 특성들)을 하나의 묶음으로 처리하여 특성의 차원을 효과적으로 줄입니다.

#### 코드 예제

먼저 LightGBM 라이브러리를 설치해야 합니다.
`!pip install lightgbm`

In [ ]:
!pip install lightgbm

In [10]:
import lightgbm as lgb

# LightGBM 모델 초기화 및 학습
lgbm_reg = lgb.LGBMRegressor(
    n_estimators=500,
    learning_rate=0.1,
    num_leaves=31,         # 하나의 트리가 가질 수 있는 최대 리프 수. default: 31
    max_depth=-1,          # 깊이 제한 없음 (num_leaves로 제어)
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

lgbm_reg.fit(X_train, y_train,
             eval_set=[(X_test, y_test)],
             eval_metric='rmse',
             callbacks=[lgb.early_stopping(50, verbose=False)])

# 예측 및 성능 평가
y_pred_lgbm = lgbm_reg.predict(X_test)
rmse_lgbm = np.sqrt(mean_squared_error(y_test, y_pred_lgbm))
r2_lgbm = r2_score(y_test, y_pred_lgbm)

print(f"[LightGBM Regressor 성능]")
print(f"RMSE: {rmse_lgbm:.4f}")
print(f"R^2 Score: {r2_lgbm:.4f}")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000673 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1313
[LightGBM] [Info] Number of data points in the train set: 7008, number of used features: 23
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 704.767837

[LightGBM Regressor 성능]
RMSE: 141.3432
R^2 Score: 0.9521


*Note: LightGBM에서는 `max_depth`보다 `num_leaves`가 모델의 복잡도를 제어하는 더 중요한 파라미터입니다. `num_leaves`는 $2^{max\_depth}$ 보다 작게 설정하는 것이 일반적입니다.*


#### 🎯 연습문제 5

LightGBM의 `num_leaves` 파라미터를 `10`으로 줄였을 때와 `100`으로 늘렸을 때, 모델의 RMSE와 학습 시간은 어떻게 변하는지 비교해보세요. (학습 시간은 `%timeit` 매직 커맨드를 사용하거나, `time` 모듈을 활용하여 측정할 수 있습니다.)


#### 🎯 연습문제 6

LightGBM이 XGBoost에 비해 가지는 가장 큰 장점인 '리프 중심 트리 분할' 방식이 '수준 중심 트리 분할' 방식과 어떻게 다른지, 그리고 이 차이가 어떤 결과를 가져오는지 설명해보세요.


### 3. CatBoost (Categorical Boosting)

CatBoost는 Yandex에서 개발한 부스팅 라이브러리로, 이름에서 알 수 있듯이 **범주형(Categorical) 특성**을 처리하는 데 매우 강력한 성능을 보여줍니다.

#### CatBoost의 주요 특징

  * **Ordered Target Statistics**: 범주형 특성을 수치형으로 변환하는 가장 큰 문제점은 **타겟 누수(Target Leakage)**입니다. 즉, 변환 과정에서 타겟 변수의 정보가 특성에 포함되어 과적합을 유발합니다. CatBoost는 데이터를 임의의 순서로 섞고, 각 샘플의 범주형 특성을 인코딩할 때 해당 샘플보다 앞에 있는 데이터들의 타겟 통계값(평균 등)만을 사용하여 이 문제를 해결합니다. 이것이 바로 Ordered TS입니다.
  * **자동 범주형 특성 처리**: 사용자가 어떤 특성이 범주형인지 `cat_features` 파라미터에 지정해주기만 하면, CatBoost가 내부적으로 최적의 방식으로 처리합니다. 원-핫 인코딩 같은 번거로운 전처리 과정이 필요 없습니다.
  * **대칭 트리(Symmetric Trees)**: 모든 노드가 동일한 조건으로 분할되는 대칭적인(oblivious) 트리를 사용하여 예측 속도가 매우 빠르고 과적합을 방지하는 데 효과적입니다.

#### 코드 예제

먼저 CatBoost 라이브러리를 설치해야 합니다.
`!pip install catboost`

In [ ]:
!pip install catboost

In [13]:
import catboost as cb

# 원-핫 인코딩 전 데이터 다시 준비
df_orig = pd.read_csv('../datasets/ml/bike-sharing/SeoulBikeData.csv', encoding='cp949')

df_orig.columns = ['Date', 'Rented_Bike_Count', 'Hour', 'Temperature', 'Humidity',
                   'Wind_Speed', 'Visibility', 'Dew_Point_Temp', 'Solar_Radiation',
                   'Rainfall', 'Snowfall', 'Seasons', 'Holiday', 'Functioning_Day']
df_orig = df_orig.drop('Date', axis=1)
df_orig.head()

,Rented_Bike_Count,Hour,Temperature,Humidity,Wind_Speed,Visibility,Dew_Point_Temp,Solar_Radiation,Rainfall,Snowfall,Seasons,Holiday,Functioning_Day
0,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
1,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
2,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes
3,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
4,78,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes


In [14]:
# 범주형 특성 지정
categorical_features = ['Seasons', 'Holiday', 'Functioning_Day']

# 특성과 타겟, 학습/테스트 데이터 분리
X_cat = df_orig.drop('Rented_Bike_Count', axis=1)
y_cat = df_orig['Rented_Bike_Count']
X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(X_cat, y_cat, test_size=0.2, random_state=42)

In [15]:
# CatBoost 모델 초기화 및 학습
cat_reg = cb.CatBoostRegressor(
    iterations=500,         # n_estimators와 동일
    learning_rate=0.1,
    depth=6,                # max_depth와 동일
    cat_features=categorical_features, # 범주형 특성 명시
    random_state=42,
    verbose=0               # 학습 과정 출력 안 함
)
cat_reg.fit(X_train_cat, y_train_cat)

In [16]:
# 예측 및 성능 평가
y_pred_cat = cat_reg.predict(X_test_cat)
rmse_cat = np.sqrt(mean_squared_error(y_test_cat, y_pred_cat))
r2_cat = r2_score(y_test_cat, y_pred_cat)

print(f"[CatBoost Regressor 성능]")
print(f"RMSE: {rmse_cat:.4f}")
print(f"R^2 Score: {r2_cat:.4f}")

[CatBoost Regressor 성능]
RMSE: 226.0760
R^2 Score: 0.8773


#### 🎯 연습문제 7

CatBoost 모델을 학습시킬 때 `cat_features` 파라미터를 지정하지 않으면 어떻게 될까요? `cat_features`를 지정했을 때와 지정하지 않았을 때의 성능(RMSE)을 비교하고, 왜 그런 차이가 발생하는지 설명해보세요. (힌트: 지정하지 않으면 CatBoost는 해당 특성을 수치형으로 간주합니다.)

#### 🎯 연습문제 8

만약 '도시명'처럼 매우 많은 고유값을 가지는 범주형 특성(High-Cardinality Feature)이 있다면, 원-핫 인코딩 방식의 문제점은 무엇이며, CatBoost가 이를 어떻게 효과적으로 해결할 수 있는지 설명해보세요.


-----

## Ⅲ. 종합 실습 과제: 따릉이 수요 예측 모델 성능 비교 🏆

지금까지 배운 Gradient Boosting 모델 3대장(XGBoost, LightGBM, CatBoost)을 모두 사용하여 '서울 따릉이' 수요 예측 문제를 해결해보고, 최종 성능을 비교 분석하여 최적의 모델을 찾아봅시다.

### 과제 목표

1.  간단한 특성 공학(Feature Engineering)을 수행합니다.
2.  XGBoost, LightGBM, CatBoost 모델을 각각 학습시키고 성능(RMSE, R²)을 평가합니다.
3.  세 모델의 성능을 시각적으로 비교하고 결과를 해석합니다.

### Step 1: 데이터 준비 및 특성 공학

기존에 사용한 `df_orig` 데이터프레임에 새로운 시간 관련 특성을 추가해 봅시다.

In [ ]:
# 원본 데이터 로드

# 컬럼명 정리

# 📌 특성 공학: 주중/주말, 출퇴근 시간 특성 추가

# 사용할 특성 정의

# 범주형 특성 목록

# 학습/테스트 데이터 분리

### Step 2: 3개 모델 학습 및 평가

아래 가이드에 따라 XGBoost, LightGBM, CatBoost 모델을 각각 학습시키고, 결과를 `results` 리스트에 저장하세요.

  * **XGBoost**: `pd.get_dummies`로 범주형 변수를 원-핫 인코딩한 데이터를 사용해야 합니다.
  * **LightGBM**: `pd.get_dummies`로 인코딩된 데이터를 사용하거나, `pandas.Categorical` 타입으로 변환하여 직접 처리할 수 있습니다. 여기서는 XGBoost와 동일하게 인코딩된 데이터를 사용하겠습니다.
  * **CatBoost**: 원-핫 인코딩되지 않은 원본 형태의 데이터를 사용하고, `cat_features` 인자를 꼭 설정해주세요.

<!-- end list -->

In [ ]:
# 결과 저장을 위한 리스트

# --- 1. XGBoost ---
# XGBoost를 위한 데이터 전처리 (원-핫 인코딩)

# 모델 학습

# 성능 계산 및 저장

# --- 2. LightGBM ---
# LightGBM은 XGBoost와 동일한 데이터 사용

# 성능 계산 및 저장


# --- 3. CatBoost ---
# CatBoost는 원본 데이터 사용

# 성능 계산 및 저장

# 결과 데이터프레임 생성

### Step 3: 결과 비교 및 시각화

`plotly.express`를 사용하여 세 모델의 RMSE와 R² 점수를 막대그래프로 비교해 봅시다.

In [ ]:
# RMSE 비교 시각화

# R² Score 비교 시각화

### Step 4: 결과 해석 (자율 양식)

  * 어떤 모델이 가장 좋은 성능(가장 낮은 RMSE, 가장 높은 R²)을 보였나요?
  * 각 모델의 학습 시간은 어땠나요? (코드를 수정하여 측정해보세요.)
  * 이 데이터셋의 특성(범주형 변수의 존재, 데이터 크기 등)을 고려할 때, 왜 특정 모델이 더 좋은 성능을 보였을지 자신의 생각을 정리해보세요.
  * 기타 생각해볼 점이 있다면 적어보세요.